In [23]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [32]:
df_movies = pd.read_csv("../processed_data/movies.csv")
df_movies.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,tagline,title,...,spoken_languages_list,production_companies_list,production_countries_list,release_year,return,director,transformed_title,transformed_director,release_month,release_day
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,NaN,Toy Story,...,['en'],['Pixar Animation Studios'],['US'],1995,12.45,John Lasseter,toystory,johnlasseter,octubre,lunes
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Roll the dice and unleash the excitement!,Jumanji,...,"['en', 'fr']","['TriStar Pictures', 'Teitler Film', 'Intersco...",['US'],1995,4.04,Joe Johnston,jumanji,joejohnston,diciembre,viernes
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,...,['en'],"['Warner Bros.', 'Lancaster Gate']",['US'],1995,0.00,Howard Deutch,grumpieroldmen,howarddeutch,diciembre,viernes
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Friends are the people who let you be yourself...,Waiting to Exhale,...,['en'],['Twentieth Century Fox Film Corporation'],['US'],1995,5.09,Forest Whitaker,waitingtoexhale,forestwhitaker,diciembre,viernes
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,...,['en'],"['Sandollar Productions', 'Touchstone Pictures']",['US'],1995,0.00,Charles Shyer,fatherofthebridepartii,charlesshyer,febrero,viernes


In [6]:
tfidf = TfidfVectorizer(stop_words="english")

In [22]:
df_movies["overview"].fillna("", inplace=True)
tfidf_matrix = tfidf.fit_transform(df_movies["overview"])

In [25]:
cosine_sin = linear_kernel(tfidf_matrix, tfidf_matrix) 

MemoryError: Unable to allocate 18.5 GiB for an array with shape (49815, 49815) and data type float64

In [ ]:
index = pd.Series(df_movies.index, df_movies["title"]).drop_duplicates()

In [ ]:
def get_recommendations(title, cosine_sin = cosine_sin):
    idx = index[title]
    sim_scores = enumerate(cosine_sin[idx])
    sim_scores = sorted(sim_scores,key = lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]

    sim_index = [i[0]for i in sim_scores]
    print(df_movies["title"].iloc[sim_index])

get_recommendations('The Dark Knight Rises')